# <br> Собираем данные в python </center>


## Agenda 

* Азы всех азов
* Что делать, если сервер разозлился
* Что такое API 
* Что такое Selenium 
* Хитрости

# 1. Азы всех азов

## Зачем собирать данные автоматически? 

<br>

<br>

<center>
<img src="https://raw.githubusercontent.com/hse-econ-data-science/eds_spring_2020/master/sem05_parsing/image/aaaaaa.png" width="500"> 

## Что такое HTML? 

**HTML (HyperText Markup Language)**  — это такой же язык разметки как Markdown или LaTeX. Он является стандартным для написания различных сайтов. Команды в таком языке называются **тегами**. Если открыть абсолютно любой сайт, нажать на правую кнопку мышки, а после нажать `View page source`, то перед вами предстанет HTML скелет этого сайта.

HTML-страница это ни что иное, как набор вложенных тегов. Можно заметить, например, следующие теги:

- `<title>` – заголовок страницы
- `<h1>…<h6>` – заголовки разных уровней
- `<p>` – абзац (paragraph)
- `<div>` – выделения фрагмента документа с целью изменения вида содержимого
- `<table>` – прорисовка таблицы 
- `<tr>` – разделитель для строк в таблице 
- `<td>` – разделитель для столбцов в таблице
- `<b>` – устанавливает жирное начертание шрифта

Обычно команда `<...>` открывает тег, а  `</...>` закрывает его. Все, что находится между этими двумя командами, подчиняется правилу, которое диктует тег. Например, все, что находится между `<p>` и  `</p>` — это отдельный абзац.   

Теги образуют своеобразное дерево с корнем в теге `<html>` и разбивают страницу на разные логические кусочки. У каждого тега есть свои потомки (дети) — те теги, которые вложены в него и свои родители. 

Например, HTML-древо страницы может выглядеть вот так:


````
<html>
<head> Заголовок </head>
<body>
    <div>
        Первый кусок текста со своими свойствами
    </div>
    <div>
        Второй кусок текста
            <b>
                Третий, жирный кусок
            </b>
    </div>
    Четвёртый кусок текста
</body>
</html>
````

Можно работать с этим html как с текстом, а можно как с деревом. Обход этого дерева и есть парсинг веб-страницы. Мы всего лишь будем находить нужные нам узлы среди всего этого разнообразия и забирать из них информацию!

<center>
<img src="https://raw.githubusercontent.com/hse-econ-data-science/eds_spring_2020/master/sem05_parsing/image/tree.png" width="450"> 

## Качаем цены на книги

* Хотим собрать [цены на книги](http://books.toscrape.com)
* Руками долго, напишем код на петухоне

Доступ к веб-станицам позволяет получать модуль requests. Подгрузим его. Если у вас не установлен этот модуль, то придётся напрячься и установить:  `!pip install requests`.

In [1]:
import requests  

url = 'http://books.toscrape.com/catalogue/page-1.html'
response = requests.get(url)
response

<Response [200]>

Благословенный 200 ответ - соединение установлено и данные получены, всё чудесно! Если попытаться перейти на несуществующую страницу, то можно получить, например, знаменитую ошибку 404.

In [2]:
requests.get('http://books.toscrape.com/big_scholarship')

<Response [404]>

Внутри response лежит html-разметка странички, которую мы парсим. 

In [3]:
response.content[:1000]

b'\n\n<!DOCTYPE html>\n<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->\n<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->\n<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->\n<!--[if gt IE 8]><!--> <html lang="en-us" class="no-js"> <!--<![endif]-->\n    <head>\n        <title>\n    All products | Books to Scrape - Sandbox\n</title>\n\n        <meta http-equiv="content-type" content="text/html; charset=UTF-8" />\n        <meta name="created" content="24th Jun 2016 09:30" />\n        <meta name="description" content="" />\n        <meta name="viewport" content="width=device-width" />\n        <meta name="robots" content="NOARCHIVE,NOCACHE" />\n\n        <!-- Le HTML5 shim, for IE6-8 support of HTML elements -->\n        <!--[if lt IE 9]>\n        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>\n        <![endif]-->\n\n        \n            <link rel="shortcut icon"

Выглядит неудобоваримо, как насчет сварить из этого дела что-то покрасивее? Например, прекрасный суп.

<img align="center" src="https://raw.githubusercontent.com/hse-econ-data-science/eds_spring_2020/master/sem05_parsing/image/alisa.jpg" height="200" width="200"> 

Пакет **[`bs4`](https://www.crummy.com/software/BeautifulSoup/)**, a.k.a **BeautifulSoup** был назван в честь стишка про красивый суп из Алисы в стране чудес. Эта совершенно волшебная библиотека, которая из сырого и необработанного HTML (или XML) кода страницы выдаст вам структурированный массив данных, по которому очень удобно искать необходимые теги, классы, атрибуты, тексты и прочие элементы веб страниц.

> Пакет под названием `BeautifulSoup` — скорее всего, не то, что вам нужно. Это третья версия (*Beautiful Soup 3*), а мы будем использовать четвертую. Так что нам нужен пакет `beautifulsoup4`. Чтобы было совсем весело, при импорте нужно указывать другое название пакета — `bs4`, а импортировать функцию под названием `BeautifulSoup`. В общем, сначала легко запутаться, но эти трудности нужно преодолеть однажды, а потом будет проще.

In [4]:
#!pip install bs4

In [5]:
from bs4 import BeautifulSoup

# распарсили страничку в дерево 
tree = BeautifulSoup(response.content, 'html.parser')

Внутри переменной `tree` теперь лежит дерево из тегов, по которому мы можем совершенно спокойно бродить. 

In [6]:
tree.html.head.title

<title>
    All products | Books to Scrape - Sandbox
</title>

Можно вытащить из того места, куда мы забрели, текст с помощью метода `text`.

In [7]:
tree.html.head.title.text

'\n    All products | Books to Scrape - Sandbox\n'

С текстом можно работать классическими питоновскими методами. Например, можно избавиться от лишних отступов.

In [8]:
tree.html.head.title.text.strip()

'All products | Books to Scrape - Sandbox'

Для каждой книги основная информация находится внутри тега `article`, для которого прописан класс `product_pod` (грубо говоря, в html класс задаёт оформление соотвествующего кусочка страницы). Вытащим инфу о книге из этого тега. 

In [9]:
tree.find('article', {'class': 'product_pod'}).find('p', {'class': 'price_color'}).text

'£51.77'

In [10]:
books = tree.find_all('article', {'class' : 'product_pod'})
books[0]

<article class="product_pod">
<div class="image_container">
<a href="a-light-in-the-attic_1000/index.html"><img alt="A Light in the Attic" class="thumbnail" src="../media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg"/></a>
</div>
<p class="star-rating Three">
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
</p>
<h3><a href="a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a></h3>
<div class="product_price">
<p class="price_color">£51.77</p>
<p class="instock availability">
<i class="icon-ok"></i>
    
        In stock
    
</p>
<form>
<button class="btn btn-primary btn-block" data-loading-text="Adding..." type="submit">Add to basket</button>
</form>
</div>
</article>

In [11]:
type(books)

bs4.element.ResultSet

In [12]:
len(books)

20

Полученный после поиска объект также обладает структурой bs4. Поэтому можно продолжить искать нужные нам объекты уже в нём.

In [13]:
type(books[0])

bs4.element.Tag

In [14]:
books[0].find('p', {'class': 'price_color'}).text

'£51.77'

Обратите внимание, что для поиска есть как минимум два метода: `find` и `find_all`. Если несколько элементов на странице обладают указанным адресом, то метод `find` вернёт только самый первый. Чтобы найти все элементы с таким адресом, нужно использовать метод `find_all`. На выход будет выдан список.

Кроме содержимого у тегов часто есть атрибуты. Например, у названия книги есть атрибуты `title` и `href`: 

In [15]:
books[0].h3

<h3><a href="a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a></h3>

Их тоже можно вытащить.

In [16]:
books[0].h3.a.get('href')

'a-light-in-the-attic_1000/index.html'

In [17]:
books[0].h3.a.get('title')

'A Light in the Attic'

А ещё по этим атрибутам можно искать интересующие нас кусочки страницы. 

In [18]:
tree.find_all('a', {'title': 'A Light in the Attic'})

[<a href="a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a>]

Собственно говоря, это всё. 

Обратите внимание, что на сайте все книги лежат на разных страничках. Если попробовать потыкать их, можно заметить, что в ссылке будет меняться атрибут `page`. Значит, если мы хотим собрать все книги, надо создать кучу ссылок с разным `page` внутри цикла. Когда качаешь данные с более сложных сайтов, в ссылке часто есть огромное количество атрибутов, которые регулируют выдачу.


Давайте запишем весь код для сбора книг в виде функции. На вход она будет принимать номер странички, которую надо скачать. 

In [19]:
def get_page(p):
    
    # изготовили ссылку
    url = 'http://books.toscrape.com/catalogue/page-{}.html'.format(p)
    
    # сходили по ней
    response = requests.get(url)
    
    # построили дерево 
    tree = BeautifulSoup(response.content, 'html.parser')
    
    # нашли в нём всё самое интересное
    books = tree.find_all('article', {'class' : 'product_pod'})
    
    infa = [ ]
    
    for book in books:
        infa.append({'price': book.find('p', {'class': 'price_color'}).text,
                     'href': book.h3.a.get('href'),
                     'title': book.h3.a.get('title')})
                     
    return infa

Осталось только пройтись по всем страничкам от page-1 до page-50 циклом и данные у нас в кармане. 

In [20]:
a = []
a.extend([1, 2])

In [21]:
a

[1, 2]

In [22]:
[1, 2] + [3, 4]

[1, 2, 3, 4]

In [23]:
infa = []

for p in range(1,51):
    infa.extend(get_page(p))

In [24]:
len(infa)

1000

In [25]:
infa[:10]

[{'price': '£51.77',
  'href': 'a-light-in-the-attic_1000/index.html',
  'title': 'A Light in the Attic'},
 {'price': '£53.74',
  'href': 'tipping-the-velvet_999/index.html',
  'title': 'Tipping the Velvet'},
 {'price': '£50.10',
  'href': 'soumission_998/index.html',
  'title': 'Soumission'},
 {'price': '£47.82',
  'href': 'sharp-objects_997/index.html',
  'title': 'Sharp Objects'},
 {'price': '£54.23',
  'href': 'sapiens-a-brief-history-of-humankind_996/index.html',
  'title': 'Sapiens: A Brief History of Humankind'},
 {'price': '£22.65',
  'href': 'the-requiem-red_995/index.html',
  'title': 'The Requiem Red'},
 {'price': '£33.34',
  'href': 'the-dirty-little-secrets-of-getting-your-dream-job_994/index.html',
  'title': 'The Dirty Little Secrets of Getting Your Dream Job'},
 {'price': '£17.93',
  'href': 'the-coming-woman-a-novel-based-on-the-life-of-the-infamous-feminist-victoria-woodhull_993/index.html',
  'title': 'The Coming Woman: A Novel Based on the Life of the Infamous Femin

In [26]:
import pandas as pd

df = pd.DataFrame(infa)
print(df.shape)
df.head()

(1000, 3)


,price,href,title
0,£51.77,a-light-in-the-attic_1000/index.html,A Light in the Attic
1,£53.74,tipping-the-velvet_999/index.html,Tipping the Velvet
2,£50.10,soumission_998/index.html,Soumission
3,£47.82,sharp-objects_997/index.html,Sharp Objects
4,£54.23,sapiens-a-brief-history-of-humankind_996/index...,Sapiens: A Brief History of Humankind


In [27]:
df.to_csv('parsed_books.csv', index=False)
df.to_excel('parsed_books.xlsx', index=False)

Кстати говоря, если перейти по ссылке в саму книгу, там о ней будет куча дополнительной информации. Можно пройтись по всем ссылкам и выкачать себе по ним дополнительную информацию. 

# 2. Что делать, если сервер разозлился

* Вы решили собрать себе немного данных 
* Сервер не в восторге от ковровой бомбардировки автоматическими запросами 
* Error 403, 404, 504, $\ldots$ 
* Капча, требования зарегистрироваться
* Заботливые сообщения, что с вашего устройства обнаружен подозрительный трафик

<center>
<img src="https://raw.githubusercontent.com/hse-econ-data-science/eds_spring_2020/master/sem05_parsing/image/doge.jpg" width="450"> 

## а) быть терпеливым 

* Слишком частые запросы раздражают сервер
* Ставьте между ними временные задержки 

In [28]:
import time
time.sleep(3) # и пусть весь мир подождёт 3 секунды

## б) быть похожим на человека


Запрос нормального человека через браузер выглядит так: 

<center>
<img src="https://raw.githubusercontent.com/hse-econ-data-science/eds_spring_2020/master/sem05_parsing/image/browser_get.png" width="600"> 
    
С ним на сервер попадает куча информации! Запрос от питона выглядит так: 


<center>
<img src="https://raw.githubusercontent.com/hse-econ-data-science/eds_spring_2020/master/sem05_parsing/image/python_get.jpg" width="250"> 
 
Заметили разницу?  Очевидно, что нашему скромному запросу не тягаться с таким обилием мета-информации, которое передается при запросе из обычного браузера. К счастью, никто нам не мешает притвориться человечными и пустить пыль в глаза сервера при помощи генерации фейкового юзер-агента. Библиотек, которые справляются с такой задачей, существует очень и очень много, лично мне больше всего нравится [fake-useragent.](https://pypi.org/project/fake-useragent/) При вызове метода из различных кусочков будет генерироваться рандомное сочетание операционной системы, спецификаций и версии браузера, которые можно передавать в запрос:

In [29]:
import warnings
warnings.filterwarnings("ignore")

In [30]:
from fake_useragent import UserAgent
UserAgent().chrome

'Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.93 Safari/537.36'

Например, https://knowyourmeme.com/ не захочет пускать к себе python и выдаст ошибку 403. Она выдается сервером, если он доступен и способен обрабатывать запросы, но по некоторым личным причинам отказывается это делать.

In [31]:
url = 'https://knowyourmeme.com/'

response = requests.get(url)
response

<Response [403]>

А если сгенерировать User-Agent, вопросов у сервера не возникнет. 

In [32]:
response = requests.get(url, headers={'User-Agent': UserAgent().chrome})
response

<Response [200]>

__Другой пример:__ если захотите спарсить ЦИАН, он начнет вам выдавать капчу. Один из вариантов обхода: менять ip через тор. Однако на практически каждый запрос из-под тора, ЦИАН будет выдавать капчу. Если добавить в запрос `User_Agent`, то капча будет вылезать намного реже. 

## в) общаться через посредников

<center>
<img src="https://raw.githubusercontent.com/hse-econ-data-science/eds_spring_2020/master/sem05_parsing/image/proxy.jpeg" width="400"> 

Посмотрим на свой ip-адрес без прокси. 

In [33]:
r = requests.get('https://httpbin.org/ip')
print(r.json())

{'origin': '83.220.238.93'}


А теперь попробуем посмотреть, что будет если подключить прокси.

In [34]:
proxies = {
    'http': '182.53.206.47:47592',
    'https': '182.53.206.47:47592'
}

r = requests.get('https://httpbin.org/ip', proxies=proxies)

print(r.json())

ProxyError: HTTPSConnectionPool(host='httpbin.org', port=443): Max retries exceeded with url: /ip (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f876e561460>: Failed to establish a new connection: [Errno 61] Connection refused')))

Запрос работал немного подольше, ip адрес сменился. Большая часть проксей, которые вы найдёте работают криво. Иногда запрос идёт очень долго и выгоднее сбросить его и попробовать другую проксю. Это можно настроить опцией `timeout`.  Например, так если сервер не будет отвечать секунду, код упадёт. 

In [35]:
import requests
requests.get('http://www.google.com', timeout=1)

<Response [200]>

__Где можно попытаться раздобыть списки прокси:__ 

* https://getfreeproxylists.blogspot.com/
* Большая часть бесплатных прокси обычно не работает.

## г) уходить глубже 

<center>
<img src="https://raw.githubusercontent.com/hse-econ-data-science/eds_spring_2020/master/sem05_parsing/image/tor.jpg" width="600"> 

Можно попытаться обходить злые сервера через тор. Есть аж несколько способов, но мы про это говорить не будем. Лучше подробно почитать в нашей статье на Хабре. Ссылка на неё в конце тетрадки. Ещё в самом начале была. А ещё в середине [наверняка есть.](https://habr.com/ru/company/ods/blog/346632/)

## Совместить всё? 

1. Начните с малого 
2. Если продолжает банить, накидывайте новые примочки
3. Каждая новая примочка бьёт по скорости 

# 5. Хитрости: 

### Хитрость 1:  Не стесняйтесь пользоваться `try-except`

Эта конструкция позволяет питону в случае ошибки сделать что-нибудь другое либо проигнорировать её. Например, мы хотим найти логарифм от всех чисел из списка: 

In [36]:
from math import log 

a = [1,2,3,-1,-5,10,3]

for item in a:
    print(log(item))

0.0
0.6931471805599453
1.0986122886681098


ValueError: math domain error

У нас не выходит, так как логарифм от отрицательных чисел не берётся. Чтобы код не падал при возникновении ошибки, мы можем его немного изменить: 

In [37]:
from math import log 

a = [1,2,3,-1,-5,10,3]

for item in a:
    try:
        print(log(item))  # попробуй взять логарифм
    except:
        print('я не смог') # если не вышло, сознайся и работай дальше

0.0
0.6931471805599453
1.0986122886681098
я не смог
я не смог
2.302585092994046
1.0986122886681098


__Как это использовать при парсинге?__  Интернет создаёт человек. У многих людей руки очень кривые. Предположим, что мы на ночь поставили парсер скачивать цены, он отработал час и упал из-за того, что на како-нибудь одной странице были криво проставлены теги, либо вылезло какое-то редкое поле, либо вылезли какие-то артефакты от старой версии сайта, которые не были учтены в нашем парсере. Гораздо лучше, чтобы код проигнорировал эту ошибку и продолжил работать дальше. 

### Хитрость 2:  pd.read_html

Если на странице, которую вы спарсили, среди тэгов `<tr>` и `<td>` прячется таблица, чаще всего можно забрать её себе без написания цикла, который будет перебирать все стобцы и строки. Поможет в этом `pd.read_html`. Например, вот так можно забрать себе [табличку с сайта ЦБ](https://cbr.ru/currency_base/daily/) 

In [38]:
import pandas as pd

df = pd.read_html('https://cbr.ru/currency_base/daily/')[0]
df.head()

,Цифр. код,Букв. код,Единиц,Валюта,Курс
0,36,AUD,1,Австралийский доллар,397651
1,944,AZN,1,Азербайджанский манат,333589
2,51,AMD,100,Армянских драмов,133341
3,933,BYN,1,Белорусский рубль,221152
4,975,BGN,1,Болгарский лев,301553


In [39]:
df.shape

(34, 5)

In [40]:
df.tail()

,Цифр. код,Букв. код,Единиц,Валюта,Курс
29,203,CZK,10,Чешских крон,238368
30,752,SEK,10,Шведских крон,557890
31,756,CHF,1,Швейцарский франк,587426
32,710,ZAR,10,Южноафриканских рэндов,356228
33,392,JPY,100,Японских иен,425752


Команда пытается собрать в массив все таблички c веб-страницы. Если хочется, можно сначала через bs4 найти нужную таблицу, а потом уже распарсить её: 

In [41]:
resp = requests.get('https://cbr.ru/currency_base/daily/')
tree = BeautifulSoup(resp.content, 'html.parser')

# нашли табличку
table = tree.find_all('table', {'class' : 'data'})[0]

# распарсили её
df = pd.read_html(str(table))[0]
df.head()

,Цифр. код,Букв. код,Единиц,Валюта,Курс
0,36,AUD,1,Австралийский доллар,397651
1,944,AZN,1,Азербайджанский манат,333589
2,51,AMD,100,Армянских драмов,133341
3,933,BYN,1,Белорусский рубль,221152
4,975,BGN,1,Болгарский лев,301553


### Хитрость 3:  используйте пакет tqdm

> Код уже работает час. Я вообще без понятия когда он закончит работу. Было бы круто узнать, сколько ещё ждать... 

Если в вашей голове возникла такая мысль, пакет `tqdm` ваш лучший друг. Установите его: ```pip install tqdm```

In [42]:
import time
from tqdm import tqdm_notebook

a = list(range(20))

# 30 раз будем спать по секунде
for i in tqdm_notebook(a):
    time.sleep(1)

Мы обмотали тот вектор, по которому идёт цикл в `tqdm_notebook`. Это даёт нам красивую зелёную строку, которая показывает насколько сильно мы продвинулись по коду. Обматывайте свои самые большие и долгие циклы в `tqdm_notebook` и всегда понимайте сколько осталось до конца. 

### Хитрость 4:  распараллеливание

Если сервер не очень настроен вас банить, можно распаралелить свои запросы к нему. Самый простой способ сделать это — библиотека `joblib`. 

In [43]:
from joblib import Parallel, delayed
from tqdm import tqdm_notebook

def simple_function(x):
    return x**2


nj = -1 # паралель на все ядра 
result = Parallel(n_jobs=nj)(
                delayed(simple_function)(item)          # какую функцию применяем 
                for item in tqdm_notebook(range(10)))   # к каким объектам применям

# tqdm_notebook в последней строчке будет создавать зелёный бегунок с прогрессом

На самом деле это не самый эффективный способ паралелить в python. Он жрёт много памяти и работает медленнее, чем [стандартный multiprocessing.](https://docs.python.org/3/library/multiprocessing.html) Но зато две строчки, КАРЛ! Две строчки! 

###  Ещё хитрости: 

* __Сохраняйте то, что парсите по мере скачки!__ Прямо внутрь цикла запихните код, который сохраняет файл! 
* Когда код упал в середине списка для скачки, не обязательно запускать его с самого начала. Просто сохраните тот кусок, который уже скачался и дозапустите код с места падения.
* Засовывать цикл для обхода ссылок внутрь функции - не самая хорошая идея. Предположим, что надо обойти $100$ ссылок. Функция должна вернуть на выход объекты, которые скачались по всему этому добру. Она берёт и падает на $50$ объекте. Конечно же то, что уже было скачано, функция не возвращает. Всё, что вы накачали - вы теряете. Надо запускать заново. Почему? Потому что внутри функции своё пространство имён. Если бы вы делали это циклом влоб, то можно было бы сохранить первые $50$ объектов, которые уже лежат внутри листа, а потом продолжить скачку. 
* Не ленитесь листать документацию. Из неё можно узнать много полезных штук. 

# 6. Почиташки

* [Парсим мемы в python](https://habr.com/ru/company/ods/blog/346632/) - подробная статья на Хабре, по которой можно научиться ... парсить (ВНЕЗАПНО) 
* [Тетрадки Ильи Щурова](https://github.com/ischurov/pythonhse) про python для анализа данных. В [лекции 9](https://nbviewer.jupyter.org/github/ischurov/pythonhse/blob/master/Lecture%209.ipynb) и [лекции 10](https://nbviewer.jupyter.org/github/ischurov/pythonhse/blob/master/Lecture%2010.ipynb) есть про парсеры. 


# Задание на семинаре

С сайта https://quotes.toscrape.com/ собрать текст цитаты, автора и теги. Сохранить в таблицу.

In [44]:
import requests
from bs4 import BeautifulSoup

Шаг 1 – научиться собирать цитаты для одной страницы

In [48]:
### your code here

Шаг 1.1 – разобраться как из html для одной цитаты тянуть автора и текст цитаты

In [ ]:
### your code here

Шаг 1.2 Соберем инфу для одной страницы!

In [ ]:
### your code here

Шаг 2. Сделать это все для всех страниц

In [ ]:
### your code here